# Análisis del Exponente de Hurst en Econofísica

## Teoría del Exponente de Hurst (H)

### Definición
El **exponente de Hurst** es una medida estadística que caracteriza el comportamiento de autocorrelación de una serie de tiempo. Fue introducido por Harold Hurst en 1951 para analizar niveles de agua en el río Nilo.

### Método de Cálculo: Análisis R/S (Range/Standard Deviation)

El análisis R/S se basa en la siguiente metodología:

1. **Dividir la serie** en sub-series de tamaño $L$
2. **Para cada sub-serie**, calcular:
   - **R (Rango)**: $R = \max(Y) - \min(Y)$, donde $Y$ es la serie de incrementos acumulados
   - **S (Desviación estándar)**: desviación estándar de los retornos
3. **R/S promedio** para cada escala $L$
4. **Regresión log-log**: $\log(R/S) = H \log(L) + c$

El **exponente de Hurst (H)** es la pendiente de esta regresión.

### Interpretación del Exponente H

$$H = \begin{cases}
H = 0.5 & \text{Caminata Aleatoria (Ruido Blanco)} \\
H > 0.5 & \text{Persistencia (Trending) - Comportamiento tendencial} \\
H < 0.5 & \text{Antipersistencia (Mean-Reverting) - Reversión a la media}
\end{cases}$$

#### a) **H ≈ 0.5: Caminata Aleatoria (Random Walk)**
- Cambios independientes del pasado
- **Modelo**: Mercado eficiente (Hipótesis de Mercado Eficiente - EMH)
- **Predicción**: No hay patrones predecibles
- **Implicación**: Imposible obtener ganancias anormales
- **Ejemplo**: Retornos de acciones bajo EMH

#### b) **H > 0.5: Persistencia (Trending Behavior)**
- Movimientos pasados **refuerzan** la dirección actual
- Si sube hoy, es más probable que siga subiendo mañana
- **Comportamiento**: Tendencias, momentum
- **Predicción**: Parcialmente predecible
- **Implicación**: Estrategias de momentum pueden funcionar
- **Rango típico**: 0.5 < H < 1.0 (H = 1 es la caminata pura)
- **Ejemplo**: Precios de commodities, tendencias a largo plazo

#### c) **H < 0.5: Antipersistencia (Mean-Reverting)**
- Movimientos pasados se **revierten**
- Si sube, es más probable que baje después
- **Comportamiento**: Volatilidad, fluctuaciones
- **Predicción**: Reversión predecible
- **Implicación**: Estrategias de reversal pueden funcionar
- **Ejemplo**: Retornos de bonos, ajustes de mercado

### Significado Financiero

| H | Comportamiento | Riesgo | Oportunidad |
|---|---|---|---|
| 0.5 | Eficiente | Riesgo puro | Ninguna |
| > 0.5 | Trending | Riesgo amplificado | Momentum |
| < 0.5 | Mean-revert | Riesgo reducido | Arbitraje |

### Críticas y Limitaciones

1. **Sensibilidad a la ventana**: Varía según el período analizado
2. **No estacionariedad**: Los datos financieros pueden cambiar propiedades en el tiempo
3. **Efecto de tamaño**: Diferentes horizontes de tiempo dan diferentes H
4. **Ruido estadístico**: Con datos limitados, H es inestable
5. **Suposiciones**: Assume auto-similitud en diferentes escalas

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# ============================================================================
# ANÁLISIS DEL EXPONENTE DE HURST (H)
# 
# Aplicación de econofísica para detectar comportamiento de series de tiempo:
# ¿Son los mercados financieros "caminatas aleatorias" (eficientes)?
# ¿O tienen patrones predecibles (trending o mean-reversion)?
# ============================================================================

def hurst(ts):
    """
    Calcula el exponente de Hurst mediante el método R/S (Range/Standard Deviation).
    
    TEORÍA:
    -------
    El análisis R/S mide la dependencia de largo plazo en una serie.
    
    Proceso:
    1. Dividir la serie de tiempo en N sub-series de tamaño L
    2. Para cada sub-serie:
       a) Calcular la media de la sub-serie
       b) Calcular desviaciones acumuladas: Y(i) = sum(x_j - mean)
       c) Rango: R = max(Y) - min(Y)
       d) Desv. estándar: S = std(x_j)
       e) Ratio R/S para esta ventana
    3. Promediar R/S sobre todas las ventanas para cada L
    4. Iterar sobre diferentes tamaños L (desde pequeño a grande)
    5. Regresión log-log: log(R/S) = H * log(L) + c
    
    La pendiente H es el EXPONENTE DE HURST
    
    INTERPRETACIÓN DE RESULTADOS:
    =============================
    H = 0.5 : Caminata aleatoria (eficiente, independencia estadística)
    H > 0.5 : Persistencia (trending, momentum, autocorrelación positiva)
    H < 0.5 : Antipersistencia (mean-reversion, sobreajuste, autocorrelación negativa)
    
    ARGS:
        ts : array-like, serie de tiempo para analizar
        
    RETURNS:
        H : float, exponente de Hurst
    """
    ts = np.array(ts)
    N = len(ts)
    
    # Requiere datos suficientes para estabilidad estadística
    if N < 50:
        return np.nan

    # Crear rango de tamaños de ventana (lag)
    # Usamos desde escala pequeña (2) a escala grande (N/4)
    # Rango logarítmico permite detectar comportamiento multi-escala
    lags = np.arange(2, N // 4)
    if len(lags) < 2: 
        return np.nan
    
    RS_log = []      # log(R/S) para cada escala
    lag_log = []     # log(L) para cada escala

    # Iterar sobre diferentes escalas de tiempo
    for L in lags:
        # Dividir serie en ventanas no superpuestas de tamaño L
        num_windows = N // L
        sub_series = ts[:num_windows * L].reshape(num_windows, L)

        RS_per_window = []
        
        for window in sub_series:
            # --- Calcular R/S para esta ventana ---
            
            # Media de la ventana
            media_ventana = np.mean(window)
            
            # Desviaciones respecto a la media
            desviaciones = window - media_ventana
            
            # Serie de incrementos acumulados
            # (Camino que resultaría si sumamos desviaciones)
            Y = np.cumsum(desviaciones)
            
            # Rango: diferencia entre máximo y mínimo del camino acumulado
            # Representa cuán lejos viaja la serie
            R = np.max(Y) - np.min(Y)
            
            # Desviación estándar: mide dispersión de los retornos
            S = np.std(window)
            
            # Ratio R/S normalizados (solo si S > 0 para evitar división por cero)
            if S > 1e-6:
                RS_per_window.append(R / S)
            
        # Promediar R/S para todos los windows de tamaño L
        if RS_per_window:
            RS_promedio = np.mean(RS_per_window)
            
            # Transformación log para regresión
            # log(R/S) se plotea vs log(L)
            RS_log.append(np.log(RS_promedio))
            lag_log.append(np.log(L))

    if not RS_log:
        return np.nan

    # REGRESIÓN LOG-LOG
    # =================
    # Ajustamos línea: log(R/S) = H * log(L) + constante
    # La pendiente H es el exponente de Hurst
    H, _ = np.polyfit(lag_log, RS_log, 1)
    return H

# ============================================================================
# GENERACIÓN DE SERIES DE TIEMPO PARA ANÁLISIS
# ============================================================================

# Generar datos simulados en lugar de descargarlos
np.random.seed(42)
n_obs = 1260  # ~5 años de datos diarios

# --- 1. RETORNOS CON PERSISTENCIA (Trending) ---
# Agregamos tendencia lineal a retornos aleatorios
# Esto simula un mercado donde hay momentum (H > 0.5)

retornos_base = np.random.randn(n_obs) * 0.02
tendencia = np.linspace(0, 0.01, n_obs)  # Tendencia creciente débil
returns_spy = retornos_base + tendencia

print("Serie 1: Retornos con Trending (persistencia)")
print(f"  - Contiene ruido aleatorio + tendencia")
print(f"  - Esperamos H > 0.5 (persistencia)")

# --- 2. PRECIO CON MOVIMIENTO BROWNIANO TENDENCIAL (GLD - Oro) ---
# Oro a menudo muestra comportamiento de largo plazo más tendencial
# Simulamos con cambios correlacionados

ruido_gld = np.random.randn(n_obs) * 0.01
prices_gld = 100 + np.cumsum(ruido_gld + 0.0005)

print("\nSerie 2: Precios con Movimiento Browniano Tendencial")
print(f"  - Cambios acumulativos (random walk + drift)")
print(f"  - Esperamos H ≈ 1.0 (caminata aleatoria pura)")

# --- 3. RUIDO BLANCO (Benchmark: H = 0.5) ---
# Cambios completamente independientes
# Esto sirve como BASELINE para mercados eficientes

ruido_blanco = np.random.randn(n_obs)

print("\nSerie 3: Ruido Blanco (Baseline H = 0.5)")
print(f"  - Cambios completamente independientes")
print(f"  - Representa mercado eficiente bajo EMH")
print(f"  - Esperamos H ≈ 0.5")

# --- 4. CAMINATA ALEATORIA PURA (Benchmark: H = 0.5) ---
# Integral de ruido blanco
# Cambios acumulados sin estructura

caminata_aleatoria = np.cumsum(ruido_blanco)

print("\nSerie 4: Caminata Aleatoria Pura (Benchmark H = 0.5)")
print(f"  - Suma acumulada de cambios independientes")
print(f"  - Proceso de Wiener (Movimiento Browniano)")
print(f"  - Esperamos H ≈ 0.5")

# ============================================================================
# CÁLCULO DE EXPONENTES DE HURST
# ============================================================================

print("\n" + "="*70)
print("CALCULANDO EXPONENTES DE HURST...")
print("="*70)

H_returns_spy = hurst(returns_spy)
H_prices_gld = hurst(prices_gld)
H_ruido_blanco = hurst(ruido_blanco)
H_caminata_aleatoria = hurst(caminata_aleatoria)

# ============================================================================
# ANÁLISIS E INTERPRETACIÓN
# ============================================================================

print("\n" + "="*70)
print("RESULTADOS: ANÁLISIS DEL EXPONENTE DE HURST (H)")
print("="*70)

def discutir_H(H, nombre_serie):
    """
    Interpreta el exponente de Hurst en contexto financiero.
    """
    if np.isnan(H):
        print(f"\n❌ {nombre_serie}: Insuficientes datos o problemas de cálculo.")
        return
    
    print(f"\n📊 {nombre_serie}")
    print(f"   Exponente H = {H:.4f}")
    print("   " + "-"*60)
    
    if H > 0.5:
        desviacion = H - 0.5
        print(f"   ✓ PERSISTENCIA (Trending Behavior)")
        print(f"   Desviación: +{desviacion:.4f} sobre eficiencia")
        print(f"")
        print(f"   TEORÍA: La serie tiene MEMORIA A LARGO PLAZO")
        print(f"   - Si la serie sube hoy → más probable que siga subiendo")
        print(f"   - Si baja hoy → más probable que siga bajando")
        print(f"   - Los cambios pasados refuerzan la dirección actual")
        print(f"")
        print(f"   MERCADOS: Indica INEFICIENCIA")
        print(f"   - Existen patrones predecibles")
        print(f"   - Estrategias de momentum pueden tener ventaja")
        print(f"   - Riesgo: Trends pueden persistir creando burbujas")
        
    elif H < 0.5:
        desviacion = 0.5 - H
        print(f"   ✓ ANTIPERSISTENCIA (Mean-Reverting)")
        print(f"   Desviación: -{desviacion:.4f} bajo eficiencia")
        print(f"")
        print(f"   TEORÍA: La serie tiende a REVERTIRSE")
        print(f"   - Si la serie sube → más probable que baje después")
        print(f"   - Si baja → más probable que suba después")
        print(f"   - Cambios se cancelan entre sí")
        print(f"")
        print(f"   MERCADOS: Indica sobre-reacción")
        print(f"   - Excesiva volatilidad")
        print(f"   - Estrategias de reversal pueden funcionar")
        print(f"   - Riesgo: Flash crashes por sobreajuste")
        
    else:  # H ≈ 0.5
        print(f"   ✓ CAMINATA ALEATORIA (Random Walk / Eficiente)")
        print(f"")
        print(f"   TEORÍA: La serie es IMPREDECIBLE")
        print(f"   - Cambios son independientes del pasado")
        print(f"   - No hay correlación temporal (autocorrelación = 0)")
        print(f"")
        print(f"   MERCADOS: Hipótesis de Mercado Eficiente (EMH)")
        print(f"   - No hay patrones explotables")
        print(f"   - Precio refleja toda la información disponible")
        print(f"   - Imposible obtener ganancias anormales de largo plazo")

# Ejecutar análisis
discutir_H(H_returns_spy, 'Retornos de SPY')
discutir_H(H_prices_gld, 'Precio de GLD')
discutir_H(H_ruido_blanco, 'Ruido Blanco (Teórico)')
discutir_H(H_caminata_aleatoria, 'Caminata Aleatoria (Teórico)')

# ============================================================================
# RESUMEN
# ============================================================================

print("\n" + "="*70)
print("CONCLUSIONES Y APLICACIONES")
print("="*70)

print("""
1. BENCHMARKS TEÓRICOS:
   - Ruido Blanco y Caminata Aleatoria DEBEN tener H ≈ 0.5
   - Desviaciones = errores de muestreo o no-estacionariedad

2. DATOS REALES:
   - Retornos: Típicamente H ∈ [0.48, 0.55] (cercano a eficiente)
   - Precios: Típicamente H > 0.5 (trending)
   - Volatilidad: Típicamente H > 0.5 (clustering de volatilidad)

3. IMPLICACIONES PARA ESTRATEGIAS:
   - H > 0.55 → Momentum puede funcionar
   - H < 0.45 → Mean-reversion puede funcionar
   - H ≈ 0.5 → Seguir índices pasivos es óptimo

4. LIMITACIONES:
   - H varía con el período de análisis
   - Requiere muchos datos para estabilidad
   - Datos no estacionarios pueden dar resultados espurios
   - No es predictor único, usar en combinación con otros
""")

print("="*70)

Serie 1: Retornos con Trending (persistencia)
  - Contiene ruido aleatorio + tendencia
  - Esperamos H > 0.5 (persistencia)

Serie 2: Precios con Movimiento Browniano Tendencial
  - Cambios acumulativos (random walk + drift)
  - Esperamos H ≈ 1.0 (caminata aleatoria pura)

Serie 3: Ruido Blanco (Baseline H = 0.5)
  - Cambios completamente independientes
  - Representa mercado eficiente bajo EMH
  - Esperamos H ≈ 0.5

Serie 4: Caminata Aleatoria Pura (Benchmark H = 0.5)
  - Suma acumulada de cambios independientes
  - Proceso de Wiener (Movimiento Browniano)
  - Esperamos H ≈ 0.5

CALCULANDO EXPONENTES DE HURST...



RESULTADOS: ANÁLISIS DEL EXPONENTE DE HURST (H)

📊 Retornos de SPY
   Exponente H = 0.5866
   ------------------------------------------------------------
   ✓ PERSISTENCIA (Trending Behavior)
   Desviación: +0.0866 sobre eficiencia

   TEORÍA: La serie tiene MEMORIA A LARGO PLAZO
   - Si la serie sube hoy → más probable que siga subiendo
   - Si baja hoy → más probable que siga bajando
   - Los cambios pasados refuerzan la dirección actual

   MERCADOS: Indica INEFICIENCIA
   - Existen patrones predecibles
   - Estrategias de momentum pueden tener ventaja
   - Riesgo: Trends pueden persistir creando burbujas

📊 Precio de GLD
   Exponente H = 1.0047
   ------------------------------------------------------------
   ✓ PERSISTENCIA (Trending Behavior)
   Desviación: +0.5047 sobre eficiencia

   TEORÍA: La serie tiene MEMORIA A LARGO PLAZO
   - Si la serie sube hoy → más probable que siga subiendo
   - Si baja hoy → más probable que siga bajando
   - Los cambios pasados refuerzan la dire